# Automated ML with Regression model 

# On case of Taxi Data



<img src="flow2.png"
     alt="Flow"
     style="float: left; margin-right: 10px;" />

In [1]:
from azureml.opendatasets import NycTlcGreen
import pandas as pd
from datetime import datetime
from dateutil.relativedelta import relativedelta

### Getting Taxi Data

In [2]:
green_taxi_df = pd.DataFrame([])
start = datetime.strptime("1/1/2015","%m/%d/%Y")
end = datetime.strptime("1/31/2015","%m/%d/%Y")

for sample_month in range(12):
    temp_df_green = NycTlcGreen(start + relativedelta(months=sample_month), end + relativedelta(months=sample_month)) \
        .to_pandas_dataframe()
    green_taxi_df = green_taxi_df.append(temp_df_green.sample(2000))

green_taxi_df.head(10)

ActivityStarted, to_pandas_dataframe
ActivityStarted, to_pandas_dataframe_in_worker
Target paths: ['/puYear=2015/puMonth=1/']
Looking for parquet files...
Reading them into Pandas dataframe...
Reading green/puYear=2015/puMonth=1/part-00175-tid-3144095708418575143-7b1fb9db-f589-44c3-b42e-832eaf9f427c-21996.c000.snappy.parquet under container nyctlc
Done.
ActivityCompleted: Activity=to_pandas_dataframe_in_worker, HowEnded=Success, Duration=7378.52 [ms]
ActivityCompleted: Activity=to_pandas_dataframe, HowEnded=Success, Duration=7389.27 [ms]
ActivityStarted, to_pandas_dataframe
ActivityStarted, to_pandas_dataframe_in_worker
Target paths: ['/puYear=2015/puMonth=2/']
Looking for parquet files...
Reading them into Pandas dataframe...
Reading green/puYear=2015/puMonth=2/part-00007-tid-3144095708418575143-7b1fb9db-f589-44c3-b42e-832eaf9f427c-21828.c000.snappy.parquet under container nyctlc
Done.
ActivityCompleted: Activity=to_pandas_dataframe_in_worker, HowEnded=Success, Duration=15136.95 [ms]


,vendorID,lpepPickupDatetime,lpepDropoffDatetime,passengerCount,tripDistance,puLocationId,doLocationId,pickupLongitude,pickupLatitude,dropoffLongitude,...,paymentType,fareAmount,extra,mtaTax,improvementSurcharge,tipAmount,tollsAmount,ehailFee,totalAmount,tripType
697369,2,2015-01-02 18:33:34,2015-01-02 18:57:07,1,8.33,None,None,-73.988960,40.689507,-73.998520,...,2,26.5,1.0,0.5,0.3,0.0,0.0,NaN,28.3,1.0
704409,2,2015-01-02 18:37:39,2015-01-02 18:55:51,2,6.41,None,None,-73.969635,40.693150,-73.913841,...,2,21.0,1.0,0.5,0.3,0.0,0.0,NaN,22.8,1.0
870522,1,2015-01-06 11:32:59,2015-01-06 11:41:17,1,1.30,None,None,-73.862030,40.730469,-73.843826,...,2,7.5,0.0,0.5,0.3,0.0,0.0,NaN,8.3,1.0
388462,1,2015-01-22 02:35:45,2015-01-22 02:39:46,1,1.00,None,None,-73.844368,40.721306,-73.858849,...,2,5.0,0.5,0.5,0.3,0.0,0.0,NaN,6.3,1.0
1380891,2,2015-01-27 20:31:12,2015-01-27 20:37:17,4,1.47,None,None,-73.953934,40.790741,-73.948357,...,1,7.0,0.5,0.5,0.3,1.0,0.0,NaN,9.3,1.0
250140,2,2015-01-14 00:11:54,2015-01-14 00:15:56,1,0.97,None,None,-73.890923,40.746822,-73.877335,...,2,5.5,0.5,0.5,0.3,0.0,0.0,NaN,6.8,1.0
1391352,2,2015-01-28 08:51:38,2015-01-28 08:59:18,1,1.22,None,None,-73.952324,40.803349,-73.961525,...,2,7.0,0.0,0.5,0.3,0.0,0.0,NaN,7.8,1.0
853866,2,2015-01-05 22:12:20,2015-01-05 22:22:56,1,2.52,None,None,-73.951675,40.713951,-73.949776,...,1,10.5,0.5,0.5,0.3,1.2,0.0,NaN,13.0,1.0
316693,2,2015-01-15 14:06:24,2015-01-15 14:24:09,1,5.97,None,None,-73.937569,40.810364,-73.868462,...,1,20.5,0.0,0.5,0.3,0.0,0.0,NaN,21.3,1.0
1001133,2,2015-01-17 20:21:17,2015-01-17 20:51:10,1,4.44,None,None,-73.958061,40.721779,-73.977356,...,1,22.5,0.5,0.5,0.3,2.0,0.0,NaN,25.8,1.0


## Creating different Time Various frames

In [3]:
def build_time_features(vector):
    pickup_datetime = vector[0]
    month_num = pickup_datetime.month
    day_of_month = pickup_datetime.day
    day_of_week = pickup_datetime.weekday()
    hour_of_day = pickup_datetime.hour

    return pd.Series((month_num, day_of_month, day_of_week, hour_of_day))

green_taxi_df[["month_num", "day_of_month","day_of_week", "hour_of_day"]] = green_taxi_df[["lpepPickupDatetime"]].apply(build_time_features, axis=1)
green_taxi_df.head(10)

,vendorID,lpepPickupDatetime,lpepDropoffDatetime,passengerCount,tripDistance,puLocationId,doLocationId,pickupLongitude,pickupLatitude,dropoffLongitude,...,improvementSurcharge,tipAmount,tollsAmount,ehailFee,totalAmount,tripType,month_num,day_of_month,day_of_week,hour_of_day
697369,2,2015-01-02 18:33:34,2015-01-02 18:57:07,1,8.33,None,None,-73.988960,40.689507,-73.998520,...,0.3,0.0,0.0,NaN,28.3,1.0,1,2,4,18
704409,2,2015-01-02 18:37:39,2015-01-02 18:55:51,2,6.41,None,None,-73.969635,40.693150,-73.913841,...,0.3,0.0,0.0,NaN,22.8,1.0,1,2,4,18
870522,1,2015-01-06 11:32:59,2015-01-06 11:41:17,1,1.30,None,None,-73.862030,40.730469,-73.843826,...,0.3,0.0,0.0,NaN,8.3,1.0,1,6,1,11
388462,1,2015-01-22 02:35:45,2015-01-22 02:39:46,1,1.00,None,None,-73.844368,40.721306,-73.858849,...,0.3,0.0,0.0,NaN,6.3,1.0,1,22,3,2
1380891,2,2015-01-27 20:31:12,2015-01-27 20:37:17,4,1.47,None,None,-73.953934,40.790741,-73.948357,...,0.3,1.0,0.0,NaN,9.3,1.0,1,27,1,20
250140,2,2015-01-14 00:11:54,2015-01-14 00:15:56,1,0.97,None,None,-73.890923,40.746822,-73.877335,...,0.3,0.0,0.0,NaN,6.8,1.0,1,14,2,0
1391352,2,2015-01-28 08:51:38,2015-01-28 08:59:18,1,1.22,None,None,-73.952324,40.803349,-73.961525,...,0.3,0.0,0.0,NaN,7.8,1.0,1,28,2,8
853866,2,2015-01-05 22:12:20,2015-01-05 22:22:56,1,2.52,None,None,-73.951675,40.713951,-73.949776,...,0.3,1.2,0.0,NaN,13.0,1.0,1,5,0,22
316693,2,2015-01-15 14:06:24,2015-01-15 14:24:09,1,5.97,None,None,-73.937569,40.810364,-73.868462,...,0.3,0.0,0.0,NaN,21.3,1.0,1,15,3,14
1001133,2,2015-01-17 20:21:17,2015-01-17 20:51:10,1,4.44,None,None,-73.958061,40.721779,-73.977356,...,0.3,2.0,0.0,NaN,25.8,1.0,1,17,5,20


### Remove unneeded columns and clean data

In [4]:
columns_to_remove = ["lpepPickupDatetime", "lpepDropoffDatetime", "puLocationId", "doLocationId", "extra", "mtaTax",
                     "improvementSurcharge", "tollsAmount", "ehailFee", "tripType", "rateCodeID",
                     "storeAndFwdFlag", "paymentType", "fareAmount", "tipAmount"
                    ]
for col in columns_to_remove:
    green_taxi_df.pop(col)

green_taxi_df.head(5)

,vendorID,passengerCount,tripDistance,pickupLongitude,pickupLatitude,dropoffLongitude,dropoffLatitude,totalAmount,month_num,day_of_month,day_of_week,hour_of_day
697369,2,1,8.33,-73.988960,40.689507,-73.998520,40.604721,28.3,1,2,4,18
704409,2,2,6.41,-73.969635,40.693150,-73.913841,40.743767,22.8,1,2,4,18
870522,1,1,1.30,-73.862030,40.730469,-73.843826,40.721153,8.3,1,6,1,11
388462,1,1,1.00,-73.844368,40.721306,-73.858849,40.719604,6.3,1,22,3,2
1380891,2,4,1.47,-73.953934,40.790741,-73.948357,40.776630,9.3,1,27,1,20


### Check some descriptive statistics

In [5]:
green_taxi_df.describe()

,vendorID,passengerCount,tripDistance,pickupLongitude,pickupLatitude,dropoffLongitude,dropoffLatitude,totalAmount,month_num,day_of_month,day_of_week,hour_of_day
count,24000.000000,24000.000000,24000.000000,24000.000000,24000.000000,24000.000000,24000.000000,24000.000000,24000.000000,24000.00000,24000.000000,24000.000000
mean,1.781458,1.375125,2.865866,-73.809066,40.679312,-73.821726,40.683208,14.773932,6.500000,15.22800,3.249917,13.562083
std,0.413265,1.048309,2.909872,3.053640,1.683793,2.904809,1.610980,11.829004,3.452124,8.48347,1.961280,6.799981
min,1.000000,0.000000,0.000000,-74.328621,0.000000,-96.136223,0.000000,-52.800000,1.000000,1.00000,0.000000,0.000000
25%,2.000000,1.000000,1.050000,-73.959625,40.699029,-73.967745,40.700163,7.950000,3.750000,8.00000,2.000000,9.000000
50%,2.000000,1.000000,1.900000,-73.945164,40.746645,-73.945152,40.747581,11.300000,6.500000,15.00000,3.000000,15.000000
75%,2.000000,1.000000,3.632500,-73.916805,40.802251,-73.910170,40.791210,17.800000,9.250000,23.00000,5.000000,19.000000
max,2.000000,6.000000,41.700000,0.000000,41.100567,0.000000,41.100834,495.800000,12.000000,30.00000,6.000000,23.000000


### Finalize the data

In [6]:
final_df = green_taxi_df.query("pickupLatitude>=40.53 and pickupLatitude<=40.88")
final_df = final_df.query("pickupLongitude>=-74.09 and pickupLongitude<=-73.72")
final_df = final_df.query("tripDistance>=0.25 and tripDistance<31")
final_df = final_df.query("passengerCount>0 and totalAmount>0")

columns_to_remove_for_training = ["pickupLongitude", "pickupLatitude", "dropoffLongitude", "dropoffLatitude"]
for col in columns_to_remove_for_training:
    final_df.pop(col)

### Look into data

In [7]:
final_df.describe()

,vendorID,passengerCount,tripDistance,totalAmount,month_num,day_of_month,day_of_week,hour_of_day
count,23295.000000,23295.000000,23295.000000,23295.000000,23295.000000,23295.000000,23295.000000,23295.000000
mean,1.782400,1.376862,2.927914,14.774221,6.505344,15.236274,3.255248,13.571024
std,0.412623,1.050958,2.858426,10.762520,3.452949,8.471493,1.962277,6.812324
min,1.000000,1.000000,0.250000,0.110000,1.000000,1.000000,0.000000,0.000000
25%,2.000000,1.000000,1.100000,8.160000,4.000000,8.000000,2.000000,9.000000
50%,2.000000,1.000000,1.950000,11.300000,7.000000,15.000000,3.000000,15.000000
75%,2.000000,1.000000,3.700000,17.800000,10.000000,23.000000,5.000000,19.000000
max,2.000000,6.000000,30.900000,495.800000,12.000000,30.000000,6.000000,23.000000


## Configure WorkSpace

In [8]:
from azureml.core.workspace import Workspace
ws = Workspace.from_config()

### Split the data into train and test sets


In [10]:
from sklearn.model_selection import train_test_split

y_df = final_df.pop("totalAmount")
x_df = final_df

x_train, x_test, y_train, y_test = train_test_split(x_df, y_df, test_size=0.2, random_state=223)

## Automatically train a model

#### Setup the logging and specify:
Property 	Value in this tutorial 	Description
iteration_timeout_minutes 	2 	Time limit in minutes for each iteration. Reduce this value to decrease total runtime.
iterations 	20 	Number of iterations. In each iteration, a new machine learning model is trained with your data. This is the primary value that affects total run time.
primary_metric 	spearman_correlation 	Metric that you want to optimize. The best-fit model will be chosen based on this metric.
preprocess 	True 	By using True, the experiment can preprocess the input data (handling missing data, converting text to numeric, etc.)
verbosity 	logging.INFO 	Controls the level of logging.
n_cross_validations 	5 	Number of cross-validation splits to perform when validation data is not specified.

In [11]:
import logging

automl_settings = {
    "iteration_timeout_minutes": 2,
    "iterations": 20,
    "primary_metric": 'spearman_correlation',
    "preprocess": True,
    "verbosity": logging.INFO,
    "n_cross_validations": 5
}

#### Parametrize AutoConfingML

In [12]:
from azureml.train.automl import AutoMLConfig

automl_config = AutoMLConfig(task='regression',
                             debug_log='automated_ml_errors.log',
                             X=x_train.values,
                             y=y_train.values.flatten(),
                             **automl_settings)

WARNING - From /anaconda/envs/azureml_py36/lib/python3.6/site-packages/azureml/automl/core/_vendor/automl/client/core/common/tf_wrappers.py:36: The name tf.logging.set_verbosity is deprecated. Please use tf.compat.v1.logging.set_verbosity instead.

WARNING - From /anaconda/envs/azureml_py36/lib/python3.6/site-packages/azureml/automl/core/_vendor/automl/client/core/common/tf_wrappers.py:36: The name tf.logging.ERROR is deprecated. Please use tf.compat.v1.logging.ERROR instead.



### Train the automatic regression model

In [13]:
from azureml.core.experiment import Experiment
experiment = Experiment(ws, "taxi-experiment")
local_run = experiment.submit(automl_config, show_output=True)

Running on local machine
Parent Run ID: AutoML_f621fc80-8661-4268-a930-02e205843815
Current status: DatasetFeaturization. Beginning to featurize the dataset.
Current status: DatasetEvaluation. Gathering dataset statistics.
Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturizationCompleted. Completed featurizing the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.

****************************************************************************************************
DATA GUARDRAILS SUMMARY:
For more details, use API: run.get_guardrails()

TYPE:         Missing Values Imputation
STATUS:       PASSED
DESCRIPTION:  There were no missing values found in the training data.

TYPE:         High Cardinality Feature Detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and no high cardinality features were detected.

**************************************************************

## Explore the results

Explore the results of automatic training with a Jupyter widget. The widget allows you to see a graph and table of all individual run iterations, along with training accuracy metrics and metadata. Additionally, you can filter on different accuracy metrics than your primary metric with the dropdown selector.

In [15]:
from azureml.widgets import RunDetails
RunDetails(local_run).show()

A Jupyter Widget

### And retrieve the best model

In [16]:
best_run, fitted_model = local_run.get_output()
print(best_run)
print(fitted_model)

Run(Experiment: taxi-experiment,
Id: AutoML_f621fc80-8661-4268-a930-02e205843815_18,
Type: None,
Status: Completed)
RegressionPipeline(pipeline=Pipeline(memory=None,
     steps=[('datatransformer', DataTransformer(enable_feature_sweeping=None, feature_sweeping_timeout=None,
        is_onnx_compatible=None, logger=None, observer=None, task=None)), ('prefittedsoftvotingregressor', PreFittedSoftVotingRegressor(estimators=[('1', Pipeline(memory=None,
     steps=[('minmax...666666666667, 0.06666666666666667, 0.06666666666666667, 0.13333333333333333, 0.13333333333333333]))]),
          stddev=None)


### Test the best model accuracy

Use the best model to run predictions on the test data set to predict taxi fares. The function predict uses the best model and predicts the values of y, trip cost, from the x_test data set. Print the first 10 predicted cost values from y_predict.

In [17]:
y_predict = fitted_model.predict(x_test.values)
print(y_predict[:10])

[40.4651457  12.14854707 12.84290343 24.90674126  6.74718735 10.28425064
 15.77419726 16.99371797 10.78159758  8.91181421]


In [18]:
from sklearn.metrics import mean_squared_error
from math import sqrt

y_actual = y_test.values.flatten().tolist()
rmse = sqrt(mean_squared_error(y_actual, y_predict))
rmse

3.7166037477861886

### Calculating MAPE

In [19]:
sum_actuals = sum_errors = 0

for actual_val, predict_val in zip(y_actual, y_predict):
    abs_error = actual_val - predict_val
    if abs_error < 0:
        abs_error = abs_error * -1

    sum_errors = sum_errors + abs_error
    sum_actuals = sum_actuals + actual_val

mean_abs_percent_error = sum_errors / sum_actuals
print("Model MAPE:")
print(mean_abs_percent_error)
print()
print("Model Accuracy:")
print(1 - mean_abs_percent_error)

Model MAPE:
0.14167917793597037

Model Accuracy:
0.8583208220640296


From the two prediction accuracy metrics, you see that the model is fairly good at predicting taxi fares from the data set's features, typically within +- $4.00, and approximately 15% error.

The traditional machine learning model development process is highly resource-intensive, and requires significant domain knowledge and time investment to run and compare the results of dozens of models. Using automated machine learning is a great way to rapidly test many different models for your scenario.